In [0]:
# Use Unity Catalog path
ball_df = spark.read.csv("/Volumes/ipl/default/ipl/Ball_By_Ball.csv", header=True, inferSchema=True)
match_df = spark.read.csv("/Volumes/ipl/default/ipl/Match.csv", header=True, inferSchema=True)
player_df = spark.read.csv("/Volumes/ipl/default/ipl/Player.csv", header=True, inferSchema=True)
player_match_df = spark.read.csv("/Volumes/ipl/default/ipl/Player_match.csv", header=True, inferSchema=True)
team_df = spark.read.csv("/Volumes/ipl/default/ipl/Team.csv", header=True, inferSchema=True)

display(ball_df)
display(match_df)
display(player_df)



In [0]:
# Example: Clean Match data
match_df = match_df.dropna().dropDuplicates()

print("Number of matches:", match_df.count())
print("Schema of Ball Data:")
ball_df.printSchema()


In [0]:
# Clean column names
ball_df = ball_df.toDF(*[c.strip().lower() for c in ball_df.columns])
match_df = match_df.toDF(*[c.strip().lower() for c in match_df.columns])
player_df = player_df.toDF(*[c.strip().lower() for c in player_df.columns])

# Join Ball data with Match info
ball_match_df = ball_df.join(match_df, ball_df["match_id"] == match_df["match_id"], "inner")

# Join Player info (ball_df.striker → player_df.player_id)
ball_match_player_df = ball_match_df.join(
    player_df,
    ball_match_df["striker"] == player_df["player_id"],
    "left"
)

display(ball_match_player_df)



In [0]:
from pyspark.sql.functions import sum as _sum

top_batsmen = (
    ball_match_player_df.groupBy("player_name")
    .agg(_sum("runs_scored").alias("total_runs"))
    .orderBy("total_runs", ascending=False)
)

display(top_batsmen.limit(10))



In [0]:
from pyspark.sql.functions import col

# Filter out balls where out_type is not null (i.e., a wicket fell)
wickets = ball_match_player_df.filter(col("out_type").isNotNull())

# Count wickets per bowler (column is usually 'bowler')
top_bowlers = (
    wickets.groupBy("bowler")
    .count()
    .withColumnRenamed("count", "wickets")
    .orderBy("wickets", ascending=False)
)

display(top_bowlers.limit(10))


In [0]:
team_wins = match_df.groupBy("Match_Winner") \
    .count() \
    .withColumnRenamed("count", "Total_Wins") \
    .orderBy("Total_Wins", ascending=False)

display(team_wins)


In [0]:
mom = (
    match_df.groupBy("manofmach")
    .count()
    .withColumnRenamed("count", "awards")
    .orderBy("awards", ascending=False)
)

display(mom.limit(10))



In [0]:
from pyspark.sql.functions import col

toss = (
    match_df.groupBy("toss_name", "match_winner")
    .count()
    .orderBy("count", ascending=False)
)

display(toss)


In [0]:
import matplotlib.pyplot as plt

# Convert to Pandas (top 10 for plotting)
top10_pd = top_batsmen.limit(10).toPandas()

plt.figure(figsize=(12,6))
plt.bar(top10_pd['player_name'], top10_pd['total_runs'], color='skyblue')
plt.xlabel('Player Name')
plt.ylabel('Total Runs')
plt.title('Top 10 Batsmen by Total Runs')
plt.xticks(rotation=45)
plt.show()


In [0]:
top5_pd = top_batsmen.limit(5).toPandas()

plt.figure(figsize=(8,8))
plt.pie(top5_pd['total_runs'], labels=top5_pd['player_name'], autopct='%1.1f%%', startangle=140)
plt.title('Top 5 Batsmen Contribution')
plt.show()
